# Géométrie directe

Ce notebook explore le concept de **géométrie directe** en robotique en utilisant le robot **Talos** et la bibliothèque **Pinocchio**.

Nous allons visualiser comment l'effecteur du robot peut :
- Maintenir une contrainte **3D rigide** avec une sphère placée dans l’espace.
- Maintenir une contrainte **6D rigide** (position + orientation) avec un objet.


In [ ]:
import gepetuto.magic

### 1. Chargement du robot et de la visualisation
Pinocchio permet de charger des modèles sous plusieurs formats. Ici, nous utilisons un modèle URDF du robot Talos. Il est maintenu dans une petite collection de modèle, facilement accessible avec PyPI: example robot data.

In [ ]:
# %load tp1/generated/handstand_1
import time
import unittest

import example_robot_data as robex
import numpy as np
import pinocchio as pin

from schaeffler2025.meshcat_viewer_wrapper import MeshcatVisualizer, colors

robot = robex.load("talos")
NQ = robot.model.nq
NV = robot.model.nv


On utilise MeshCat pour afficher le robot. MeshCat permet un rendu WebGL, ce qui est très pratique pour des notebooks, même si ses fonctionnalités sont limitées.

In [ ]:
viz = MeshcatVisualizer(robot)


In [ ]:
viz.viewer.jupyter_cell()

La configuration d'un robot dans Pinocchio est toujours organisée en suivant la profondeur de l'arbre cinématique. Les degrés de liberté du bassin "flottant" viennent donc en premier. Les 3 premiers coefficients sont pour la translation. Puis vient la rotation sous forme d'un quaternion. On a ensuite les autres axes. Une posture valide est disponible dans robot.q0 (la configuration "half-sitting").

In [ ]:
viz.display(robot.q0)

L'organisation des axes est décrite dans le modèle:

In [ ]:
for n in robot.model.names:
    print(n)

### 2. Ajout d'un objet dans meshcat: une sphère
MeshCat permet aussi d'afficher d'autres objets non décrits dans le modèle. On précise leur placement (6D) par un vecteur xyz-quaternion.

In [ ]:
# %load tp1/generated/handstand_2

# Ajout d'une sphère rouge dans le monde
ballID = "world/ball"
viz.addSphere(ballID, 0.1, colors.red)

# Position de la sphère : coordonnées (x, y, z) + quaternion identité
q_ball = [0.5, 0.1, 1.2, 1, 0, 0, 0]
viz.applyConfiguration(ballID, q_ball)

### 3. Position initiale du robot pour "saisir" l'objet
Pour illustrer le concept de géometrie directe, essayez de choisir une configuration du robot telle que la main du robot apparaisse saisir la sphère.

In [ ]:
q0 = robot.q0.copy()
q0[12:15] = [1,2,3]
viz.display(q0)

Voici la solution si besoin.

In [ ]:
%do_not_load_snippet tp1/generated/handstand_3

### 4. Mouvement du robot avec contrainte **3D rigide**

La géométrie directe permet de calculer l'emplacement de la sphère permettant d'assurer que le rapport entre l'effecteur et l'objet reste constant.  
Mathématiquement, cela s'écrit:

$$^OEB(q) = ^OOB(q) - ^OOE = constante$$

Connaissant $^OEB$ à la configuration initiale et $^OOB(q)$ à chaque itération, on en déduit alors $^OOE$.

Remplissez le template suivant pour assurer cette constante. 

In [ ]:
idx = robot.model.getFrameId("gripper_right_fingertip_3_link")
pin.framesForwardKinematics(robot.model, robot.data, q0)
# Position of end-eff wrt world at current configuration
o_eff = robot.data.oMf[idx].translation
o_ball = q_ball[:3]  # Position of ball wrt world

delta = np.random.rand(robot.model.nq - 7) * 0.2 - 0.1
delta[:12] = 0  # Do not move the legs
for t in range(5):
    # Chose new configuration of the robot
    q = q0.copy()
    q[7:] = q0[7:] + np.sin(3.14 * t / 100.0) * delta

    # Gets the new position of the ball
    pin.framesForwardKinematics(robot.model, robot.data, q)
    print('Placement de la main: ',  robot.data.oMf[idx] )

    # Choisissez une position pour la sphère
    o_ball = np.random.rand(3)

    
    # Display new configuration for robot and ball
    viz.applyConfiguration(ballID, o_ball.tolist() + [1, 0, 0, 0])
    viz.display(q)
    time.sleep(1e-2)


Voici la solution si besoin.

In [ ]:
%do_not_load_snippet tp1/generated/handstand_4

### 5. Avec une contrainte 6D (position + orientation)

On va maintenant chercher à imposer une contrainte 6D, en trouvant une configuration qui maintienne une relation constante sur translation et rotation. Choisissez une configuration, calculer le placement de la base et d'un des effecteurs, puis déduisez le placement de la base pour que l'effecteur soit à un emplacement particulier, par exemple 0. La configuration du robot doit être changée en convertissant la matrice de rotation en quaternion :


In [ ]:
Mbasis = pin.SE3.Random()
q0[3:7] = pin.Quaternion(Mbasis.rotation).coeffs()

Voici la solution si besoin.

In [ ]:
%do_not_load_snippet tp1/generated/handstand_5